In [1]:
%reload_ext nb_black
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from sqlalchemy import create_engine
import warnings
from statsmodels.graphics.gofplots import qqplot

warnings.filterwarnings("ignore")
sns.set(style="whitegrid")

postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "lifeexpectancy"
table_name = "lifeexpectancy"

<IPython.core.display.Javascript object>

In [2]:
engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)

life_expc_df = pd.read_sql_query("select * from lifeexpectancy", con=engine)

# no need for an open connection,
# as we're only doing a single query
engine.dispose()

<IPython.core.display.Javascript object>

In [3]:
life_expc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   object 
 1   Year                             2938 non-null   int64  
 2   Status                           2938 non-null   object 
 3   Life expectancy                  2928 non-null   float64
 4   Adult Mortality                  2928 non-null   float64
 5   infant deaths                    2938 non-null   int64  
 6   Alcohol                          2744 non-null   float64
 7   percentage expenditure           2938 non-null   float64
 8   Hepatitis B                      2385 non-null   float64
 9   Measles                          2938 non-null   int64  
 10   BMI                             2904 non-null   float64
 11  under-five deaths                2938 non-null   int64  
 12  Polio               

<IPython.core.display.Javascript object>

In [4]:
# check a missingness summary
def missingness_summary(df, print_log=False, sort=None):
    missings = df.isna().mean()

    if sort == "asc":
        missings = missings.sort_values()
    elif sort == "desc":
        missings = missings.sort_values(ascending=False)

    if print_log:
        print(missings)

    return missings

<IPython.core.display.Javascript object>

In [6]:
crash_missing_report = missingness_summary(life_expc_df, print_log=True, sort="desc")

Population                         0.221920
Hepatitis B                        0.188223
GDP                                0.152485
Total expenditure                  0.076923
Alcohol                            0.066031
Income composition of resources    0.056841
Schooling                          0.055480
 BMI                               0.011572
 thinness  1-19 years              0.011572
 thinness 5-9 years                0.011572
Diphtheria                         0.006467
Polio                              0.006467
Adult Mortality                    0.003404
Life expectancy                    0.003404
under-five deaths                  0.000000
 HIV/AIDS                          0.000000
Measles                            0.000000
percentage expenditure             0.000000
infant deaths                      0.000000
Status                             0.000000
Year                               0.000000
Country                            0.000000
dtype: float64


<IPython.core.display.Javascript object>

In this dataset, `population`, `Hepatitis B`, `GDP` are our columns with missingness from 15% to 22%. Our other missing columns will be handled next

In [13]:
life_expc_df[life_expc_df["Population"].isna()]["Country"].value_counts()

Saint Vincent and the Grenadines                        16
Slovakia                                                16
Barbados                                                16
Kuwait                                                  16
Singapore                                               16
Republic of Korea                                       16
Venezuela (Bolivarian Republic of)                      16
Egypt                                                   16
Bahrain                                                 16
Oman                                                    16
Gambia                                                  16
Brunei Darussalam                                       16
Czechia                                                 16
Somalia                                                 16
The former Yugoslav republic of Macedonia               16
United Republic of Tanzania                             16
Congo                                                   

<IPython.core.display.Javascript object>

In [20]:
# for population, I will fill interpolate, then fill any remaing with zeros
life_expc_df["Population"] = life_expc_df["Population"].interpolate()
life_expc_df[life_expc_df["Population"].isna()]["Country"].value_counts()

Series([], Name: Country, dtype: int64)

<IPython.core.display.Javascript object>

In [21]:
life_expc_df.groupby("Country")["Population"].value_counts()

Country      Population
Afghanistan  257798.0      1
             284331.0      1
             293756.0      1
             327582.0      1
             2364851.0     1
                          ..
Zimbabwe     13124267.0    1
             13558469.0    1
             14386649.0    1
             15411675.0    1
             15777451.0    1
Name: Population, Length: 2934, dtype: int64

<IPython.core.display.Javascript object>